# Cruise Control GPU (Live Mirror)

This notebook mirrors your local **CPU cruise-control** data on the GPU in real-time.
It uses only log streaming (no C code changes).


In [ ]:
import torch, time, requests

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(0))

# Match C constants
SIM_DT = 0.1
TARGET_GAP = 10.0
Kp = 0.35
Kd = 0.70


In [ ]:
def compute_block(sample, device):
    row = [[
        sample['my_x'], sample['my_y'],
        sample['leader_x'], sample['leader_y'],
        sample['current_speed'], sample['leader_speed'], sample['leader_speed']
    ]]
    d = torch.tensor(row, dtype=torch.float32, device=device)
    my_x, my_y, front_x, front_y, cur_spd, front_spd, leader_spd = d.T
    gap = torch.sqrt((my_x - front_x)**2 + (my_y - front_y)**2)
    projected_error = (gap - TARGET_GAP) - (front_spd * SIM_DT)
    base = leader_spd
    damping = (front_spd - cur_spd) * Kd
    correction = projected_error * Kp
    new_speed = base + damping + correction
    new_speed = torch.maximum(new_speed, torch.zeros_like(base))
    new_speed = torch.minimum(new_speed, base + 25.0)

    # Optional: time-to-collision (TTC)
    rel = cur_spd - front_spd
    eps = 1e-6
    inf = torch.tensor(float('inf'), device=device)
    ttc = torch.where(rel > eps, gap / rel, inf)

    return gap.item(), new_speed.item(), ttc.item()


In [ ]:
# Rajdeep
# Intruder GPU model (1000x1000 grid)
GRID_SIZE = 1000
GRID_OFFSET = GRID_SIZE // 2
INTRUDER_RADIUS = 4
INTRUDER_PROB = 0.05
INTRUDER_SPEED = 40
INTRUDER_LENGTH = 10
INTRUDER_DURATION_MS = 5000

intruder_grid = torch.zeros((GRID_SIZE, GRID_SIZE), device=device)

def _grid_idx(v):
    return max(0, min(GRID_SIZE - 1, int(round(v)) + GRID_OFFSET))

def gpu_intruder_scan(sample):
    intruder_grid.zero_()
    fx = sample['my_x']
    fy = sample['my_y']
    fx_i = _grid_idx(fx)
    fy_i = _grid_idx(fy)

    # Spawn near truck with small probability
    if torch.rand((), device=device).item() > INTRUDER_PROB:
        return False

    ox = int(torch.randint(-INTRUDER_RADIUS, INTRUDER_RADIUS + 1, (1,), device=device).item())
    oy = int(torch.randint(-INTRUDER_RADIUS, INTRUDER_RADIUS + 1, (1,), device=device).item())
    ix = max(0, min(GRID_SIZE - 1, fx_i + ox))
    iy = max(0, min(GRID_SIZE - 1, fy_i + oy))

    intruder_grid[ix, iy] = 1.0

    x0 = max(0, fx_i - INTRUDER_RADIUS)
    x1 = min(GRID_SIZE, fx_i + INTRUDER_RADIUS + 1)
    y0 = max(0, fy_i - INTRUDER_RADIUS)
    y1 = min(GRID_SIZE, fy_i + INTRUDER_RADIUS + 1)

    hit = intruder_grid[x0:x1, y0:y1].sum() > 0
    return bool(hit)


## Live URL
Paste your tunnel URL here (cloudflared).
Example: `https://abc.trycloudflare.com/latest.json`


In [ ]:
LIVE_URL = "https://meter-injection-tiny-sec.trycloudflare.com/latest.json"


In [ ]:
INTRUDER_POST_URL = LIVE_URL.rsplit('/', 1)[0] + '/intruder'


In [ ]:
# Rajdeep
seq = 0
null_shown = {}
for _ in range(2000):
    try:
        r = requests.get(LIVE_URL, timeout=5)
        if r.status_code != 200 or not r.text.strip():
            print('Waiting for live JSON...')
            time.sleep(0.5)
            continue
        s = r.json()
    except Exception as e:
        print('Waiting for live JSON...', e)
        time.sleep(0.5)
        continue

    samples = []
    if isinstance(s, dict) and 'followers' in s:
        samples = list(s.get('followers', {}).values())
    else:
        samples = [s]

    if samples and all(sample.get('disconnected') for sample in samples):
        print('All followers disconnected. Stopping GPU loop.')
        for sample in samples:
            fid = int(sample.get('follower_id', 0))
            seq += 1
            try:
                requests.post(INTRUDER_POST_URL, json={
                    'seq': seq,
                    'target_id': fid,
                    'active': False,
                    'speed': 0,
                    'length': 0,
                    'duration_ms': 0,
                }, timeout=3)
            except Exception:
                pass
        break

    for sample in samples:
        fid = int(sample.get('follower_id', 0))

        if sample.get('disconnected'):
            if not null_shown.get(fid):
                print(f'NULL (leader disconnected) fid={fid}')
                null_shown[fid] = True
            seq += 1
            try:
                requests.post(INTRUDER_POST_URL, json={
                    'seq': seq,
                    'target_id': fid,
                    'active': False,
                    'speed': 0,
                    'length': 0,
                    'duration_ms': 0,
                }, timeout=3)
            except Exception:
                pass
            continue

        null_shown[fid] = False
        gap, spd, ttc = compute_block(sample, device=device)

        active = gpu_intruder_scan(sample)
        seq += 1
        try:
            requests.post(INTRUDER_POST_URL, json={
                'seq': seq,
                'target_id': fid,
                'active': bool(active),
                'speed': INTRUDER_SPEED if active else 0,
                'length': INTRUDER_LENGTH if active else 0,
                'duration_ms': INTRUDER_DURATION_MS if active else 0,
            }, timeout=3)
        except Exception:
            pass

        print(f"fid={fid} | ts={sample['ts']:.2f} | pos=({sample['my_x']:.1f},{sample['my_y']:.1f}) | speed={sample['current_speed']:.3f} | gap={sample['gap']:.3f} | new_speed={spd:.3f}")

    time.sleep(0.5)
